# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** *H*

**Names:**

* *BAFFOU Jérémy*
* *BASSETO Antoine*
* *PINTO Andrea*

---

#### Instructions

*This is a template for part 4 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [1]:
import numpy as np
import random

---

## 2.4 PageRank

### 2.4.1 Random Surfer Model

#### Exercise 2.12

In [2]:
def file_to_directed_graph(filename):
    """ Parse the adjacency list into a graph
        in the form of a dictionnary.
        
        :param filename: name of the file containing the adjacency list
        
        :return: graph in the form of a dictionnary with an array of
        adjacent nodes as the value asociated to each key
    """
    
    with open("../data/" + filename) as f:
        content = f.read().splitlines()
    
    graph = {}
    for line in content:
        c = list(map(int, line.split()))
        graph[int(c[0])] = c[1:]
        
    return graph

In [3]:
def page_rank_naive(graph, nb_iterations):
    """ Naive implementation of the page rank algorithm,
        not doing anything in particular to counteract dangling nodes
        or unconnected components in the graph.
        
        :param graph: graph to explore, in the form of a dictionnary 
        with an array of adjacent nodes as the value asociated to each key
        :param nb_iterations: the number of iterations the random walk is
        going to try to take to explore the graph
        
        :return: the score associated to each node, normalised, in the form of an array
    """
    
    scores = np.zeros(len(graph))
    current = random.choice(list(graph))
    scores[current] += 1
    
    for i in range(nb_iterations):
        
        # If the current node is a dangling one, we can just return the current result as nothing else can be done
        if not graph[current]:
            return scores / np.sum(scores)
    
        current = int(random.choice(graph[current]))
        scores[current] += 1
        
    return scores / np.sum(scores)

In [39]:
def print_info(filename, naive=False):
    """ Utility function to print some information on a given graph.
    
        :param filename: name of the file containing the adjacency list
        :param naive: whether to use the naive version of page_rank or not
    """
    graph = file_to_directed_graph(filename)
    nb_iterations = 500

    print(f"The graph : {graph}\n")

    for i in range(10):
        if naive:
            scores = page_rank_naive(graph, nb_iterations)
        else:
            scores = page_rank(graph, nb_iterations)
        print(f"trial {i} : {scores}")
        print(f"\tranking : {np.argsort(scores)}")
        print("\n")

In [34]:
print_info("absorbing.graph", naive=True)

The graph : {0: [1, 4], 1: [], 2: [3], 3: [0, 1, 2], 4: [1]}

trial 0 : [0. 1. 0. 0. 0.]
	ranking : [0 2 3 4 1]


trial 1 : [0.         0.33333333 0.33333333 0.33333333 0.        ]
	ranking : [0 4 1 2 3]


trial 2 : [0.  0.5 0.  0.5 0. ]
	ranking : [0 2 4 1 3]


trial 3 : [0.         0.33333333 0.33333333 0.33333333 0.        ]
	ranking : [0 4 1 2 3]


trial 4 : [0. 1. 0. 0. 0.]
	ranking : [0 2 3 4 1]


trial 5 : [0.         0.08333333 0.41666667 0.5        0.        ]
	ranking : [0 4 1 2 3]


trial 6 : [0.5 0.5 0.  0.  0. ]
	ranking : [2 3 4 0 1]


trial 7 : [0.   0.25 0.25 0.5  0.  ]
	ranking : [0 4 1 2 3]


trial 8 : [0.         0.33333333 0.33333333 0.33333333 0.        ]
	ranking : [0 4 1 2 3]


trial 9 : [0.  0.5 0.  0.  0.5]
	ranking : [0 2 3 1 4]




As the graph presents an absorbing node in node 1, if the random walk starts there node 1 will have a maximum score.

In general, we can see that the results are quite disparate from one trial to the other, because the presence of a dangling node comes down to having a variable number of iterations from one trial to the next, depending on how soon the random walk gets there.

In [35]:
print_info("components.graph", naive=True)

The graph : {0: [1], 1: [2, 3], 2: [0], 3: [2], 4: [5, 6], 5: [6], 6: [7], 7: [4]}

trial 0 : [0.         0.         0.         0.         0.28143713 0.15768463
 0.28143713 0.27944112]
	ranking : [0 1 2 3 5 7 4 6]


trial 1 : [0.28143713 0.28143713 0.28143713 0.15568862 0.         0.
 0.         0.        ]
	ranking : [4 5 6 7 3 0 1 2]


trial 2 : [0.28343313 0.28343313 0.28143713 0.15169661 0.         0.
 0.         0.        ]
	ranking : [4 5 6 7 3 2 0 1]


trial 3 : [0.         0.         0.         0.         0.28542914 0.14570858
 0.28343313 0.28542914]
	ranking : [0 1 2 3 5 6 4 7]


trial 4 : [0.         0.         0.         0.         0.28542914 0.14371257
 0.28542914 0.28542914]
	ranking : [0 1 2 3 5 4 6 7]


trial 5 : [0.         0.         0.         0.         0.28742515 0.13772455
 0.28742515 0.28742515]
	ranking : [0 1 2 3 5 4 6 7]


trial 6 : [0.28542914 0.28542914 0.28542914 0.14371257 0.         0.
 0.         0.        ]
	ranking : [4 5 6 7 3 0 1 2]


trial 7 : [0.283

As the graph presents two connected components, depending on where the random walk starts only one is explored. This can be observed in the above trials where there always is either the first or last four nodes with a score of 0.

Apart from this, because there are no dangling nodes, the scores for each component are quite similar from one trial to the next when they are the one being explored.

#### Exercise 2.13

In [36]:
def page_rank(graph, nb_iterations, damping_factor=0.15):
    """ Implementation of the page rank algorithm.
        
        :param graph: graph to explore, in the form of a dictionnary 
        with an array of adjacent nodes as the value asociated to each key
        :param nb_iterations: the number of iterations the random walk is
        going to try to take to explore the graph
        :param damping_factor: the probability with which the random walk with just
        start from a new random node
        
        :return: the score associated to each node, normalised, in the form of an array
    """
    scores = np.zeros(len(graph))
    current = random.choice(list(graph))
    scores[current] += 1
    
    for i in range(nb_iterations):
        if np.random.choice([True, False], p=[damping_factor, 1-damping_factor]):
            current = random.choice(list(graph))
        elif not graph[current]:
            current = random.choice(list(graph))
        else:
            current = int(random.choice(graph[current]))
        scores[current] += 1
        
    return scores / np.sum(scores)

In [37]:
print_info("absorbing.graph")

The graph : {0: [1, 4], 1: [], 2: [3], 3: [0, 1, 2], 4: [1]}

trial 0 : [0.14171657 0.31936128 0.18163673 0.23552894 0.12175649]
	ranking : [4 0 2 3 1]


trial 1 : [0.1756487  0.38922156 0.11177645 0.18962076 0.13373253]
	ranking : [2 4 0 3 1]


trial 2 : [0.13772455 0.32934132 0.15568862 0.22754491 0.1497006 ]
	ranking : [0 4 2 3 1]


trial 3 : [0.13572854 0.29341317 0.17365269 0.24351297 0.15369261]
	ranking : [0 4 2 3 1]


trial 4 : [0.15568862 0.33732535 0.15169661 0.21756487 0.13772455]
	ranking : [4 2 0 3 1]


trial 5 : [0.14770459 0.32734531 0.15568862 0.21157685 0.15768463]
	ranking : [0 2 4 3 1]


trial 6 : [0.14171657 0.32135729 0.15768463 0.22155689 0.15768463]
	ranking : [0 2 4 3 1]


trial 7 : [0.13373253 0.37125749 0.13572854 0.20558882 0.15369261]
	ranking : [0 2 4 3 1]


trial 8 : [0.16167665 0.38123752 0.13373253 0.17365269 0.1497006 ]
	ranking : [2 4 0 3 1]


trial 9 : [0.17165669 0.35329341 0.13572854 0.19760479 0.14171657]
	ranking : [2 4 0 3 1]




Explain result here

In [38]:
print_info("components.graph")

The graph : {0: [1], 1: [2, 3], 2: [0], 3: [2], 4: [5, 6], 5: [6], 6: [7], 7: [4]}

trial 0 : [0.15568862 0.14770459 0.1497006  0.06986028 0.13173653 0.06786427
 0.14770459 0.12974052]
	ranking : [5 3 7 4 1 6 2 0]


trial 1 : [0.1257485  0.1237525  0.13173653 0.06986028 0.16167665 0.08782435
 0.15369261 0.14570858]
	ranking : [3 5 1 0 2 7 6 4]


trial 2 : [0.13572854 0.13772455 0.14770459 0.06387226 0.13572854 0.08782435
 0.14770459 0.14371257]
	ranking : [3 5 0 4 1 7 2 6]


trial 3 : [0.16367265 0.17165669 0.17964072 0.10578842 0.10179641 0.0758483
 0.10578842 0.09580838]
	ranking : [5 7 4 3 6 0 1 2]


trial 4 : [0.11976048 0.11976048 0.1237525  0.06586826 0.15768463 0.09381238
 0.16167665 0.15768463]
	ranking : [3 5 0 1 2 4 7 6]


trial 5 : [0.15568862 0.13972056 0.15169661 0.07185629 0.13373253 0.06986028
 0.13772455 0.13972056]
	ranking : [5 3 4 6 1 7 2 0]


trial 6 : [0.14371257 0.14570858 0.14171657 0.06586826 0.14371257 0.08383234
 0.13972056 0.13572854]
	ranking : [3 5 7 6 2 0 

Explain result here

---

### 2.4.2 Power Iteration Method

#### Exercise 2.14: Power Iteration method

In [40]:
def file_to_google_matrix(filename, theta=0.85):
    """ Computation of the google matrix for the given graph.
        
        :param filename: name of the file containing the adjacency list
        :param theta: probability of the random walk to stay on its current
        path, 1 - theta is therefore the probability to start from a new random
        node
        
        :return: google matrix in the form of an np.array
    """
    
    with open("../data/" + filename) as f:
        content = f.read().splitlines()
    
    nb_nodes = len(content)
    g = np.zeros((nb_nodes, nb_nodes))
    
    for line in content:
        c = list(map(int, line.split()))
        outgoing_degree = len(c) - 1
        
        if outgoing_degree == 0:
            g[c[0]] = np.ones(nb_nodes) / nb_nodes
        else:
            for i in c[1:]:
                g[c[0]][i] = 1 / outgoing_degree
        
    return theta * g + (1 - theta) * np.ones((nb_nodes, nb_nodes)) / nb_nodes

In [41]:
def power_iteration(g, nb_iterations=50):
    """ Computation of the score of each node given a google matrix.
        
        :param g: google matrix
        :param nb_iterations: number of iteration used to calculate 
        the score vector
        
        :return: the score vector
    """
    
    v = np.ones(np.shape(g)[0]) / np.shape(g)[0]

    for i in range(nb_iterations):
        v = v @ g

    return v

In [42]:
def get_wikipedia_pages(id_array):
    """ Utility function to get the title of Wikipedia pages given their index
        
        :param id_array: an array containing the indexes of the pages of which we
        want the titles.
        
        :return: list of strings corresponding to the titles of the pages, given in
        the same order as in id_array
    """
    
    with open("../data/wikipedia_titles.tsv") as f:
        # Strip away the first line, because it corresponds to a legend and not to a page
        content = np.array(f.read().splitlines()[1:])
        
    # Return the selected ids and format the result to only contain page titles (and not their id)
    return list(map(lambda x: x.split(None, 1)[1], content[id_array]))

In [46]:
filename = "wikipedia.graph"
g = file_to_google_matrix(filename)
scores = power_iteration(g)
for i, t in enumerate(get_wikipedia_pages(np.argsort(scores)[-1:-11:-1])):
    print(f"rank {i + 1} : {t}")

rank 1 : United States
rank 2 : United Kingdom
rank 3 : France
rank 4 : Europe
rank 5 : Germany
rank 6 : England
rank 7 : World War II
rank 8 : Latin
rank 9 : India
rank 10 : English language


---

### 2.4.3 Gaming the system *(Bonus)*

#### Exercise 2.15 *(Bonus)*

In [47]:
def get_score_and_rank_of_page(scores, page_id):
    """ Utility function to get the score and rank of a Wikipedia page given its index
        
        :param scores: an array containing the scores of each pages
        :param page_id: the id of the page of which we want the score and rank
        
        :return: formated string with the wnated information
    """
    
    nb_of_pages = len(scores)
    
    page_title = get_wikipedia_pages([page_id])[0]
    score = scores[page_id]
    rank = np.nonzero(np.argsort(scores)[::-1] == page_id)[0][0]
    
    # Return a formated strings, with the rank going from 1 for the best to the number of pages for the worst
    return f"Page \"{page_title}\":\n\tScore: {score}\n\tRank: {rank + 1} out of {nb_of_pages}"

In [48]:
filename = "wikipedia.graph"
g = file_to_google_matrix(filename)
scores = power_iteration(g)

# ID of page "United States" as a check
page_id = 5210
print(get_score_and_rank_of_page(scores, page_id))

# ID of page "History of mathematics"
page_id = 2463
print(get_score_and_rank_of_page(scores, page_id))

Page "United States":
	Score: 0.007459087286658076
	Rank: 1 out of 5540
Page "History of mathematics":
	Score: 9.846341053223444e-05
	Rank: 2530 out of 5540


In [50]:
def cheating_google(g, scores, page_id, new_edge_budget, theta=0.85):
    """ Alteration of the given google matrix by adding new edges to boost the
        score of the given page.
        One assumption made by this algorithm is that if for one node we have, 
        for every node, a non-zero probability to go there, then it is a dangling node. This
        could be challenged by the fact that a page connected to every single page would have
        the same effect on the google matrix as a dangling node, and therefore we would misclassify
        those as dangling nodes. Because in real networks this never happens, we accept this assumption.
        
        :param g: google matrix to alter
        :param scores: scores computed with the given google matrix
        :param page_id: id of the page to boost
        :param new_edge_budget: number of new edges we are allowed to add to the 
        graph to boost the score
        :param theta: probability of the random walk to stay on its current
        path, 1 - theta is therefore the probability to start from a new random
        node, has to be the same as the one that was used to compute g in the first
        place
        
        :return: an altered google matrix
    """
    
    nb_nodes = g.shape[0]
    h_hat = g - (1 - theta) * np.ones((nb_nodes, nb_nodes)) / nb_nodes
    ranking = np.argsort(scores)[::-1]
    
    # For every best ranked pages, add an edge from there to the page we want to boost
    for i in range(new_edge_budget):
        outgoing_degree = len(np.nonzero(h_hat[ranking[i]])[0])
        
        # If the page is a dangling node (i.e. in h_hat the page is connected to all pages)
        if outgoing_degree == nb_nodes:
            # Add an edge from this dangling node to the page_id
            # i.e. the probabilty to go from this node to the selected one is 1, and to any other is 0
            line = np.zeros(nb_nodes)
            line[page_id] = 1
            h_hat[ranking[i]] = line
        else:
            # If the page is not already connected to the page we want to boost, add an edge, otherwise skip this page
            if h_hat[ranking[i]][page_id] == 0:
                h_hat[ranking[i]] *= outgoing_degree / (outgoing_degree + 1)
                h_hat[ranking[i]][page_id] += 1 / (outgoing_degree + 1)
            else:
                new_edge_budget += 1
            
    return h_hat + (1 - theta) * np.ones((nb_nodes, nb_nodes)) / nb_nodes

In [51]:
# ID of page "History of mathematics"
page_id = 2463
new_edge_budget = 300
filename = "wikipedia.graph"

g = file_to_google_matrix(filename)
scores = power_iteration(g, 50)
print("Before cheating :")
print(get_score_and_rank_of_page(scores, page_id))

print("\n")

g_cheat = cheating_google(g, scores, page_id, new_edge_budget)
scores = power_iteration(g_cheat, 50)
print(f"After cheating (by adding {new_edge_budget} new edges):")
print(get_score_and_rank_of_page(scores, page_id))

Before cheating :
Page "History of mathematics":
	Score: 9.846341053223444e-05
	Rank: 2530 out of 5540


After cheating (by adding 300 new edges):
Page "History of mathematics":
	Score: 0.005659430755510062
	Rank: 2 out of 5540
